In [1]:
import logging
import torch
import transformers
from transformers import GPT2Tokenizer, GPT2Config, GPT2Model

I0928 01:17:21.413093 140176204212032 file_utils.py:39] PyTorch version 1.0.1 available.
I0928 01:17:21.919360 140176204212032 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [2]:
print(torch.__version__, transformers.__version__)

1.0.1 2.0.0


In [3]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
vocab_file = "./pre-models/gpt2-vocab.json"
merges_file = "./pre-models/gpt2-merges.txt"

config_file = "./pre-models/gpt2-config.json"
model_file = "./pre-models/gpt2-pytorch_model.bin"

model_dir = "./pre-models"

## Encoder

In [4]:
# 激活logger 看更多信息
logging.basicConfig(level=logging.INFO)

# 加载预训练模型gpt2tokenizer (vocabulary)
tokenizer = GPT2Tokenizer(vocab_file=vocab_file, merges_file=merges_file)  # 还可以输入unk_token、bos_token、eos_token
# 编码文本
text = "Who was Jim Henson ? Jim Henson was a "
indexed_tokens = tokenizer.encode(text)

# 装换为 a PyTorch tensor
tokens_tensor = torch.tensor([indexed_tokens])
print(tokens_tensor[0])

tensor([ 8241,   373,  5395,   367, 19069,  5633,  5395,   367, 19069,   373,
          257])


## Decoder

In [5]:
# 使用 预测文本的下一个单词
# 加载本地预训练模型(weights)
config = GPT2Config.from_json_file(json_file=config_file)
model = transformers.GPT2LMHeadModel.from_pretrained(model_dir, from_tf=False, config=config)

# 设置模型的eval模式以停用dropout模块
# This is IMPORTANT to have reproductible【可繁殖？】 results during evaluation!
model.eval()

# 使用GPU
tokens_tensor = tokens_tensor.to(device)
model.to(device)

# 预测
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]

# 预测文本的下一个单词
predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])
print("预测文本：", predicted_text)
print("predicted word:", tokenizer.decode([predicted_index]))

I0928 01:17:28.144083 140176204212032 modeling_utils.py:334] loading weights file ./pre-models/pytorch_model.bin


预测文本： Who was Jim Henson? Jim Henson was a man
predicted word:  man


## 方法二